## Step 2: Zebra Falsely Mapped Read Removal
**Goal: To run [Zebra](https://github.com/biocore/zebra_filter) to remove any reads which may have been incorrectly mapped**

Citation:
Hakim D, Wandro S, Zengler K, Zaramela LS, Nowinski B, Swafford A, Zhu Q, Song SJ, Gonzalez A, McDonald D, Knight R. Zebra: Static and Dynamic Genome Cover Thresholds with Overlapping References. mSystems. 2022 Oct 26;7(5):e0075822. doi: 10.1128/msystems.00758-22

Note: Zebra coverages were run on Qiita, more info [here](https://github.com/qiita-spots/qiita/blob/ac62aba5333f537c32e213855edc39c273aa9871/CHANGELOG.md?plain=1#L9)

### Imports

In [1]:
import pandas as pd
from biom import load_table
from qiime2 import Artifact

In [2]:
#Import zebra coverages from Qiita

zebra_wol2 = pd.read_csv('qiita_downloads/qiita15336_prep16181_pangenome/188963_coverages_WoLr2/coverage_percentage.txt',
                         sep = '\t', header = None,
                         names =['gOTU', 'percent_coverage']).sort_values(by='percent_coverage', ascending = False).reset_index(drop= True)
zebra_rs210 = pd.read_csv('qiita_downloads/qiita15336_prep16181_pangenome/188968_coverages_RS210/coverage_percentage.txt',
                          sep = '\t', header = None,
                          names =['gOTU', 'percent_coverage']).sort_values(by='percent_coverage', ascending = False).reset_index(drop= True)

In [3]:
#Import qza files from SCRuB into pandas df

df_wol2 = Artifact.load('processed_data/SCRuB/qiita15336_prep16181_pangenome_wol2_scrubbed.qza').view(pd.DataFrame).T
df_rs210 = Artifact.load('processed_data/SCRuB/qiita15336_prep16181_pangenome_rs210_scrubbed.qza').view(pd.DataFrame).T

### Functions

In [4]:
def subset_coverage(df, zebra_coverage, min_coverage):
    
    #Subset zebra to at least min_coverage
    zebra_coverage = zebra_coverage[zebra_coverage['percent_coverage'] >= min_coverage]
    
    #Subset df using zebra
    df = df[df.index.isin(zebra_coverage['gOTU'])]
    
    return(df)
    

In [5]:
def combo_tech_rep(df, fn, min_coverage):
   
    #Find cols ending with '.2'
    rep_cols = [col for col in df.columns if col.endswith('.2')]

    #Create new df with paired technical replicates
    df_techReps = pd.DataFrame()
    for col in rep_cols:
        col_name = col[:-2]  #remove the '.2'
        if col_name in df.columns:
            df_techReps[col_name] = df[col_name] + df[col]
            
    #Export subset df as qza
    qza = Artifact.import_data("FeatureTable[Frequency]", df_techReps.T)
    qza.save('processed_data/Zebra_filtered/'+ fn + str(min_coverage) + '.qza')

    return(df_techReps)

### Subset biom files to having only gOTUs with at least 0.1% coverage

In [6]:
#Set min coverage percentage
min_coverage = 0.1

df_subset_wol2 = subset_coverage(df_wol2, zebra_wol2, min_coverage)
df_subset_rs210 = subset_coverage(df_rs210, zebra_rs210, min_coverage)


### Combine technical replicates and export files

In [7]:
df_subset_techRep_wol2 = combo_tech_rep(df_subset_wol2, 'qiita15336_prep16181_pangenome_wol2_scrubbed_zebraFilter', min_coverage)
df_subset_techRep_rs210 = combo_tech_rep(df_subset_rs210, 'qiita15336_prep16181_pangenome_rs210_scrubbed_zebraFilter', min_coverage)
